In [1]:
#embedding 테스트 - 완료
##one-hot으로 할 필요가 없음.
##embedding_size도 마음대로 결정할 수 있음.
##random init으로 해도됨.
##input의 embedding을 training할수도 있고 안할수도 있음
##Y는 embedding 할 필요 없음. 그냥 target class shape만 맞춰서 loss를 구하면 됨

#file에서 읽어와서 input으로 넣어보기 - 완료
##이미 index화 되어 있는 파일 읽어오기 - 완료
##dictionary file로부터 dic 생성 - 완료
##embedding 활용 - 완료

#saver 적용 - 완료

#tensorboard 적용

#padding 적용

#batch 적용

#bidirectional rnn 적용

from __future__ import print_function

import tensorflow as tf
import numpy as np
import pprint as pp
import os
import random

from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
checkpoint_path = 'data/dev/ckpt_char/'

def read_sentences_from_file(file_path):
    sentence_file = open(file_path, 'r')
    sentences = []
    while True:
        line = sentence_file.readline()
        if not line:
            break;
        sentences.append(line.replace('\n',''))
    sentence_file.close()
    return sentences

def read_sentence_indexes_from_file(file_path):
    sentence_index_file = open(file_path, 'r')
    sentence_indexes = []
    print("start reading ",file_path)
    line_count = 0
    while True:
        line = sentence_index_file.readline()
        if not line:
            break;
        sentence_indexes.append([int(c) for c in line.replace('\n','').split()])
        if line_count % 100000 == 0:
            print("reading line ",line_count)
        line_count+=1
        
    sentence_index_file.close()
    return sentence_indexes

def char_tokenizer(sentence): #sentence는 byte로 들어옴
    sentence = sentence.replace('<english>','a').replace('<number>','0')
    return [char for char in sentence] #그 후 최종 다시 byte로 변환한다

def create_dic(sentences, save_file_path):
    dic = []
    file = open(save_file_path, 'w+')
    for sentence in sentences:
        tokens = char_tokenizer(sentence)
        for token in tokens:
            if not (token in dic):
                dic.append(token)
                file.write(token+'\n')
    file.close
    return dic
    
def load_dic(dic_file_path):
    dic_file = open(dic_file_path, 'r')
    rev_vocab_dic = []
    while True:
        line = dic_file.readline()
        if not line:
            break;
        rev_vocab_dic.append(line.replace('\n',''))
    vocab_dic = {w: i for i, w in enumerate(rev_vocab_dic)}
    return vocab_dic, rev_vocab_dic

def sentences_to_indexes(sentences, vocab_dic):
    indexes = []
    for sentence in sentences:
        tokens = char_tokenizer(sentence)
        indexes.append([vocab_dic[char] for char in tokens])
    return indexes

In [3]:
target_sentence_indexes =  read_sentence_indexes_from_file('data/train/namuwiki_sentences.txt.ids10000')
target_vocab_dic, target_rev_vocab_dic = load_dic('data/dev/vocab_dev.to')

start reading  data/train/namuwiki_sentences.txt.ids10000
reading line  0
reading line  100000
reading line  200000
reading line  300000
reading line  400000
reading line  500000
reading line  600000
reading line  700000
reading line  800000
reading line  900000


In [4]:
hidden_size = 30
rnn_depth = 2
num_classes = len(target_rev_vocab_dic)
embedding_size = 30
sequence_length = 10 #window_size
learning_rate = 0.1
learning_rate_decay_rate = 0.99
batch_sentence = 100
batch_size = 256

In [5]:
def get_batch():
    dataX = []
    dataY = []
    output_weights = []

    for _ in range(batch_sentence):
        i = random.randrange(0,len(target_sentence_indexes))
        output_weight = np.ones(sequence_length, dtype=np.float32)
        target_indexes = list(target_sentence_indexes[i])
        if len(target_indexes) < sequence_length:
            continue
        
        for cursor in range(0, len(target_indexes) - sequence_length):
            x_indexes = target_indexes[cursor:cursor + sequence_length]
            y_indexes = target_indexes[cursor + 1: cursor + sequence_length + 1]
            dataX.append(x_indexes)
            dataY.append(y_indexes)
            output_weights.append(output_weight)
        
    return dataX[:batch_size], dataY[:batch_size], output_weights[:batch_size]


In [6]:
X = tf.placeholder(tf.int32, [None, sequence_length], name="input_characters")
Y = tf.placeholder(tf.int32, [None, sequence_length], name="target_characters")
weights = tf.placeholder(tf.float32, [None, sequence_length], name="sequence_weights")
learning_rate_var = tf.Variable(float(learning_rate), trainable=False, dtype=tf.float32)
learning_rate_decay_op = learning_rate_var.assign(learning_rate_var * learning_rate_decay_rate)

In [7]:
print(num_classes)
# embeddings = tf.Variable(
#     tf.random_uniform([len(target_vocab_dic), embedding_size], -1.0, 1.0), trainable=True, name="char_embeddings")
# embedded_input = tf.nn.embedding_lookup(embeddings, X, name="embedded_char_sentences")
embedded_input = tf.one_hot(X, num_classes)
print(embedded_input)

3548
Tensor("one_hot:0", shape=(?, 10, 3548), dtype=float32)


In [8]:
# Make a lstm cell with hidden_size (each unit output vector size)

def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

cell = rnn.MultiRNNCell([lstm_cell() for _ in range(rnn_depth)], state_is_tuple=True)
# cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
# cell = rnn.MultiRNNCell([cell] * rnn_depth, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, embedded_input, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size], name="reshaped_rnn_output")
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes], name="predictions")

force_weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=force_weights, name="sequence_loss")
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate_var).minimize(mean_loss)

In [9]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables()) ##saver = tf.train.Saver({"my_v2": v2})

ckpt = tf.train.get_checkpoint_state(checkpoint_path)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("Created model with fresh parameters.")
    sess.run(tf.global_variables_initializer())

Created model with fresh parameters.


In [10]:
a1,b1,c1 = get_batch()

In [ ]:
print(len(a1),len(b1),len(c1))
ccc = 4
print(a1[ccc],b1[ccc],c1[ccc])

256 256 256
[42, 3, 15, 7, 3, 453, 330, 28, 18, 3] [3, 15, 7, 3, 453, 330, 28, 18, 3, 385] [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [ ]:
step_to_test = 100
test_sentence_length = 20
loss_history = []
step_loss = 0.0
test_results = []
for i in range(10000000): ##for test
    dataX, dataY, output_weights = get_batch()
    _, l, results, label_y, learning_rate_check = sess.run(
        [train_op, mean_loss, outputs, Y, learning_rate_var], feed_dict={X: dataX, Y: dataY, weights: output_weights})
    step_loss += (l / step_to_test)
    
    if i % step_to_test == 0:
        if len(loss_history) > 2 and step_loss > max(loss_history[-3:]):
            sess.run(learning_rate_decay_op)

        loss_history.append(step_loss)
        print('step:',i,' loss:',loss_history[-1], ' learing_rate:',learning_rate_check)
        
        step_loss = 0.0
    
        for j, result in enumerate(results):
            if j > test_sentence_length:
                break
            index = np.argmax(result, axis=1)
            if j is 0:  # print all for the first result to make a sentence
                print(''.join([target_rev_vocab_dic[t] for t in index]), end='')
            else:
                print(target_rev_vocab_dic[index[-1]], end='')

        print('---------------')
        checkpoint_file_path = os.path.join(checkpoint_path, "char_rnn.ckpt")
        saver.save(sess, checkpoint_file_path)
        print('saved checkpoint')


step: 0  loss: 0.0817420768738  learing_rate: 0.1
뀔초봉탖줫땔땔땔값값봉맙맙쇀쇀똇똇똇똇십십십훡십쨌쨌쨌쎔쎔쨱---------------
saved checkpoint
step: 100  loss: 5.1945008564  learing_rate: 0.1
이                             ---------------
saved checkpoint
step: 200  loss: 4.62480917454  learing_rate: 0.1
은  aa   aa  a이 aa a이   a이    a---------------
saved checkpoint
step: 300  loss: 4.40335816383  learing_rate: 0.1
 a이 a이a이 a  a이a  a이  a이  a이   ---------------
saved checkpoint
step: 400  loss: 4.34262550831  learing_rate: 0.1
a      a    a리 고 a   a  a    a---------------
saved checkpoint
step: 500  loss: 4.23564789772  learing_rate: 0.1
 0 0   0리 기0 0이  리 0리 는 0는는   ---------------
saved checkpoint


In [ ]:
# np.argmax(test_results[0], axis=0)
np.argmax(test_results[0], axis=1)

In [ ]:
print(len(results)) #batch size
print(len(results[0])) #sequence length
print(len(results[0][0])) #vocab_size

print(batch_size)
print(sequence_length)
print(len(target_vocab_dic))


In [ ]:
#embedding 연습!!
import tensorflow as tf

embeddings = tf.Variable(
    tf.random_uniform([num_classes, 50], -1.0, 1.0))

train_inputs = tf.placeholder(tf.int32, shape=[3,2])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
ems, inputs,embd = sess.run([embeddings,train_inputs,embed], feed_dict={train_inputs:[[1,2],[2,3],[3,4]]})

for i in ems:
    print(i)
print('-----------')
print(inputs)
print('-----------')
print(embd)